In [1]:
import h5py
import shutil
from shutil import copyfile
import seaborn as sns
import glob, os
abra=os.getcwd()
import numpy as np
import matplotlib as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/ra206/miniconda3/envs/unetgpu/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [8]:
def subset_and_isolate_training_data(result_files,subsetids=[0],new_folder_name="subsetted_training_data/{}_"):
    chosen_training_data=subsetids
    for i in chosen_training_data:
        begin=result_files[i].find('session')
        strain_idx=chosen_training_data.index(i)
        filename=new_folder_name.format(result_files[i][begin:begin+22]) + "results_{:03d}.h5".format(strain_idx)
        print(filename)
        copyfile(result_files[i],filename)    

In [11]:
# Assuming you've used moseq to extract data and have a bunch of sessions 
# in a folder named {training_data_folder_name} that you would like to use to create a traing data set..

training_data_folder_name = "/n/groups/datta/Dana/longtogeny39/5_weeks/"

# assuming the folder contains all the data and you want to create a subset of that data {subsetids} to train with
# which will go into a folder called {training_data_subset_folder_name}

subsetids=[0,1]
training_data_subset_folder_name='/n/groups/datta/Dana/longtogeny39/training_set_1/'
os.mkdir(training_data_subset_folder_name)

# finally you aggregate all the training sessions into one h5 file called {final_training_data_name}

final_training_data_name='/n/groups/datta/Dana/longtogeny39/training_set_1.h5'

In [12]:
result_files=[] #create an empty matrix that will contain the addresses of the result files
for file in glob.glob(training_data_folder_name+"session*/proc/*.h5"):
    result_files.append(file)
    print(file)

/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703110809/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703150932/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703175607/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703151004/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703110659/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703175537/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703110739/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703175558/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703202215/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703150953/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703150943/proc/results_00.h5
/n/groups/datta/Dana/longtogeny39/5_weeks/session_20190703110841/proc/result

In [13]:
subset_and_isolate_training_data(result_files,subsetids=subsetids,new_folder_name=training_data_subset_folder_name)

/n/groups/datta/Dana/longtogeny39/training_set_1/results_000.h5
/n/groups/datta/Dana/longtogeny39/training_set_1/results_001.h5


In [14]:
result_files=[] #create an empty matrix that will contain the addresses of the result files
for file in glob.glob(training_data_subset_folder_name+"*.h5"):
    result_files.append(file)
    print(file)

/n/groups/datta/Dana/longtogeny39/training_set_1/results_001.h5
/n/groups/datta/Dana/longtogeny39/training_set_1/results_000.h5


In [15]:
#append all the data together into one matrix

#start with first file
data=h5py.File(result_files[0], 'r+')
frames=data['frames']
mask=data['frames_mask']

for i in range(len(result_files)):
    if i !=0:
        data=h5py.File(result_files[i], 'r+')
        next_frames=data['frames']
        next_mask=data['frames_mask']
        #print('start')
        frames=np.append(frames,next_frames,axis=0)
        print("frames " + str(i))
        mask=np.append(mask,next_mask,axis=0)
        print("mask " + str(i))
        
print('final shape of training frames ' + str(frames.shape))

frames 1
mask 1
final shape of training frames (71925, 80, 80)


In [16]:
#create and output the final h5 file

data.close()

#put new aggregated frames and mask into a file final file

f = h5py.File(final_training_data_name, "w")
dset = f.create_dataset('frames', data=frames)
dset = f.create_dataset('frames_mask', data=mask)

f.close()

In [ ]:
#shutil.rmtree('/n/groups/datta/rockwell/k2_data_gen2/subsetted_training_data/')